# Using the Gemini API: Interactive Lab
<a href="https://colab.research.google.com/github/IAT-ComputationalCreativity-Spring2025/Week10-APIs/blob/main/gemini_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this lab, we'll explore the basic usage of LLM APIs using Gemini's limited free tier

## Getting a key

If you haven't already, go to https://aistudio.google.com/apikey​ to sign in and create a new api key

Make sure not to share this key publicly

In [5]:
import os

# This will prompt for your API key but won't display what you type
from getpass import getpass

# Set environment variables
os.environ['GEMINI_API_KEY'] = getpass('Enter your Gemini API key: ')

# Verify keys are set (without revealing them)
print(f"Gemini API key is set: {'Yes' if 'GEMINI_API_KEY' in os.environ else 'No'}")

Enter your Gemini API key: ··········
Gemini API key is set: Yes


## Setting up the environment

In [1]:
# Google Gemini API
! pip install google-generativeai
# OpenAI API (optional)
#! pip install openai
# Stability AI (optional)
#! pip install stability-sdk

# For interactive widgets in Jupyter
! pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00


## Step 1: API Configuration Module

Let's create a helper module to manage API access:

In [8]:
# Try to get keys from multiple sources in order of security preference
def get_api_key(service):
    """Get API key from various sources in order of security preference"""
    if service.lower() == "gemini":
        key = os.environ.get('GEMINI_API_KEY')

        if key and key != "your_gemini_api_key_here":
            return key

        # No key found
        raise ValueError(f"No API key found for {service}. Please set up your API key using one of the methods in the notebook.")
    else:
        raise ValueError(f"Unsupported service: {service}")

## Step 2: Testing API Connection

Let's test our API connection using the Gemini API:

In [3]:
import sys
import os

# Add the current directory to path if needed
sys.path.append(os.path.abspath(""))

# Test Gemini API
import google.generativeai as genai

In [9]:
try:
    # Configure the API
    api_key = get_api_key("gemini")
    genai.configure(api_key=api_key)

    # Test a simple query
    model = genai.GenerativeModel('gemini-2.0-flash')
    response = model.generate_content("Write a haiku about artificial intelligence")

    print("API Connection Successful!")
    print("\nHaiku response:")
    print(response.text)
except Exception as e:
    print(f"Error connecting to API: {e}")
    print("\nPlease check your API key configuration and try again.")

API Connection Successful!

Haiku response:
Code learns and evolves,
Mimicking human thinking,
Future is unknown. 



## Step 3: API Rate Limit Monitoring and Error Handling

Create utility functions for API rate limiting and error handling:

In [10]:
import time
import random
from functools import wraps

class APIRateLimiter:
    def __init__(self, service_name, requests_per_minute=60):
        self.service_name = service_name
        self.min_interval = 60 / requests_per_minute  # seconds between requests
        self.last_request_time = 0
        self.request_count = 0

    def request(self, func):
        """Decorator to manage API request rates"""
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Check if we need to wait
            current_time = time.time()
            elapsed = current_time - self.last_request_time

            if elapsed < self.min_interval:
                wait_time = self.min_interval - elapsed
                print(f"Rate limiting: waiting {wait_time:.2f} seconds...")
                time.sleep(wait_time)

            # Make the request
            self.request_count += 1
            self.last_request_time = time.time()
            print(f"Making request #{self.request_count} to {self.service_name}")

            return func(*args, **kwargs)
        return wrapper


def retry_with_exponential_backoff(initial_delay=1, max_delay=60, max_retries=5):
    """Retry decorator with exponential backoff"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            delay = initial_delay
            retries = 0

            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Request failed: {e}")
                    retries += 1

                    if retries >= max_retries:
                        print(f"Maximum retries ({max_retries}) exceeded.")
                        raise

                    sleep_time = min(delay * (2 ** (retries - 1)) + random.uniform(0, 1), max_delay)
                    print(f"Retrying in {sleep_time:.2f} seconds... (Attempt {retries+1}/{max_retries})")
                    time.sleep(sleep_time)

            return func(*args, **kwargs)  # One last try
        return wrapper
    return decorator

Let's test our rate limiter and retry logic:

In [11]:
# Create rate limiters
gemini_limiter = APIRateLimiter("Gemini", requests_per_minute=60)

@gemini_limiter.request
def generate_with_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

# Test with a few requests
topics = ["water", "air", "fire"]
for i in range(min(3, len(topics))):
    try:
        result = generate_with_gemini(f"Give me one idea for a science project about {topics[i]}")
        print(f"Result: {result[:100]}...\n")
    except Exception as e:
        print(f"Error: {e}")

Making request #1 to Gemini
Result: Here's one idea for a science project about water:

**Project Idea: Testing Water Purification Metho...

Making request #2 to Gemini
Result: Here's one science project idea about air:

**Project Title:** **Does Air Pressure Affect How High a...

Making request #3 to Gemini
Result: **Investigating the Effects of Different Fuels on Fire Temperature and Flame Color.**

**Concept:** ...



In [12]:
# Test retry logic with an intentionally challenging request
@retry_with_exponential_backoff(max_retries=3)
def robust_generate_with_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

try:
    very_long_prompt = "Explain quantum computing " * 20
    result = robust_generate_with_gemini(very_long_prompt)
    print(f"\nSuccess! First 100 chars: {result[:100]}...")
except Exception as e:
    print(f"Final failure: {e}")


Success! First 100 chars: Okay, let's break down quantum computing.  Since you've asked for it explained repeatedly, I'll try ...


## Step 4: Creating a Comprehensive Helper Class

Let's create a comprehensive AI client class for our lab:

In [13]:
class AIClient:
    """A unified client for interacting with generative AI APIs"""

    def __init__(self):
        self.services = {}
        self.request_logs = []

        # Initialize available services
        self._init_gemini()

    def _init_gemini(self):
        """Initialize Gemini API if key is available"""
        try:
            api_key = get_api_key("gemini")
            if api_key:
                genai.configure(api_key=api_key)
                self.services["gemini"] = {
                    "text_model": genai.GenerativeModel('gemini-2.0-flash'),
                    "rate_limit": 60,  # requests per minute
                    "last_request": 0,
                }
                print("Gemini API initialized successfully")
        except Exception as e:
            print(f"Warning: Could not initialize Gemini API: {e}")

    def _rate_limit(self, service):
        """Apply rate limiting for the specified service"""
        if service not in self.services:
            raise ValueError(f"Service {service} not initialized")

        min_interval = 60 / self.services[service]["rate_limit"]
        current_time = time.time()
        elapsed = current_time - self.services[service]["last_request"]

        if elapsed < min_interval:
            wait_time = min_interval - elapsed
            print(f"Rate limiting {service}: waiting {wait_time:.2f} seconds...")
            time.sleep(wait_time)

        self.services[service]["last_request"] = time.time()

    def generate_text(self, prompt, service="gemini", max_retries=3):
        """Generate text from a text prompt using the specified service"""
        if service not in self.services:
            raise ValueError(f"Service {service} not available")

        for attempt in range(max_retries):
            try:
                self._rate_limit(service)

                if service == "gemini":
                    response = self.services[service]["text_model"].generate_content(prompt)

                    # Log the request
                    self.request_logs.append({
                        "service": service,
                        "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                        "timestamp": time.time(),
                        "success": True
                    })

                    return response.text
                else:
                    raise NotImplementedError(f"Text generation not implemented for {service}")

            except Exception as e:
                print(f"Error ({attempt+1}/{max_retries}): {e}")
                if attempt == max_retries - 1:
                    # Log the failed request
                    self.request_logs.append({
                        "service": service,
                        "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                        "timestamp": time.time(),
                        "success": False,
                        "error": str(e)
                    })
                    raise
                time.sleep(2 ** attempt)  # Exponential backoff

    def get_usage_stats(self):
        """Return usage statistics for all services"""
        stats = {}
        for service in self.services:
            service_logs = [log for log in self.request_logs if log["service"] == service]
            stats[service] = {
                "total_requests": len(service_logs),
                "successful_requests": len([log for log in service_logs if log["success"]]),
                "failed_requests": len([log for log in service_logs if not log["success"]]),
            }
        return stats

    def compare_responses(self, prompt, services=None):
        """Compare responses from multiple services"""
        if services is None:
            services = list(self.services.keys())

        results = {}
        for service in services:
            try:
                results[service] = self.generate_text(prompt, service=service)
            except Exception as e:
                results[service] = f"Error: {e}"

        return results

Let's test our AIClient:

In [14]:
# Create the client
client = AIClient()

# Use the client
try:
    response = client.generate_text("Explain the concept of API rate limiting in three sentences")
    print(response)

    # Check usage
    print("\nUsage Statistics:")
    print(client.get_usage_stats())
except Exception as e:
    print(f"Error using AIClient: {e}")

Gemini API initialized successfully
API rate limiting is a mechanism that controls the number of requests a user or application can make to an API within a specific timeframe. It's implemented to prevent abuse, maintain service availability, and ensure fair resource allocation for all users. By setting request limits, APIs can avoid being overwhelmed by traffic and remain responsive.


Usage Statistics:
{'gemini': {'total_requests': 1, 'successful_requests': 1, 'failed_requests': 0}}


## Step 5: An example use case -- ScientificArticleSummarizer

In [15]:
from IPython.display import Markdown, display
import ipywidgets as widgets
import time
import json

class ScientificArticleSummarizer:
    def __init__(self):
        self.history = []  # Store previous summaries

    def summarize_article(self, article_text, audience_level="general", focus_areas=None, max_length=500):
        """
        Summarizes scientific article for different audience levels with specified focus

        Parameters:
        - article_text (str): The text of the article to summarize
        - audience_level (str): "general", "undergraduate", "graduate", or "expert"
        - focus_areas (list): Specific aspects to focus on (e.g. ["methodology", "results"])
        - max_length (int): Maximum length of summary in words

        Returns:
        - str: The generated summary
        """

        # Set default focus areas if none provided
        if focus_areas is None:
            focus_areas = ["main findings", "methodology", "significance"]

        # Determine language style based on audience level
        language_style = {
            "general": "simple language avoiding technical jargon",
            "undergraduate": "introductory academic language with basic field-specific terms",
            "graduate": "advanced academic language with field-specific terminology",
            "expert": "specialized technical language appropriate for experts in the field"
        }.get(audience_level, "clear and concise language")

        # Build the prompt
        prompt = f"""
        Please summarize the following scientific article for a {audience_level} audience.
        Use {language_style}.

        Focus specifically on: {", ".join(focus_areas)}.
        Keep the summary under {max_length} words.

        Structure your response with clear sections and bullet points where appropriate.

        Article:
        {article_text[:5000]}...
        """

        if len(article_text) > 5000:
            prompt += "\n[Note: Article text was truncated due to length. This summary is based on the first portion of the article.]"

        # Generate the summary
        try:
            summary = client.generate_text(prompt)

            # Add to history
            self.history.append({
                "timestamp": time.time(),
                "audience_level": audience_level,
                "focus_areas": focus_areas,
                "summary_length": len(summary.split()),
                "summary": summary
            })

            return summary
        except Exception as e:
            return f"Error generating summary: {str(e)}"

    def extract_key_elements(self, article_text):
        """
        Extracts key elements from the article such as:
        - Main findings
        - Methodology
        - Limitations
        - Future research directions
        - Practical implications

        Returns a structured dictionary of these elements
        """

        prompt = """
        Analyze the following scientific article and extract these key elements:
        1. Main Findings: The primary results or discoveries reported
        2. Methodology: The research approach, methods, and techniques used
        3. Limitations: Any constraints, weaknesses, or limitations acknowledged
        4. Future Research: Suggested directions for further study
        5. Practical Implications: Real-world applications or consequences

        Format your response as a structured JSON object with these elements as keys.
        Keep each element concise (2-3 sentences).

        Article:
        """ + article_text[:5000]

        try:
            response = client.generate_text(prompt)

            # Try to parse the response as JSON
            # If it's not perfectly formatted JSON, we'll need to extract it
            try:
                # Look for a JSON block in the response
                import re
                json_match = re.search(r'```json\s*([\s\S]*?)\s*```', response)
                if json_match:
                    json_str = json_match.group(1)
                else:
                    json_str = response

                # Clean up and parse
                return json.loads(json_str)
            except json.JSONDecodeError:
                # If parsing fails, return the raw response
                return {"error": "Could not parse structured data", "raw_response": response}

        except Exception as e:
            return {"error": f"Error extracting key elements: {str(e)}"}

    def generate_follow_up_questions(self, article_text, num_questions=5):
        """
        Generates follow-up questions that might be asked after reading the article
        """

        prompt = f"""
        Based on the following scientific article, generate {num_questions} thoughtful follow-up questions that:
        1. Probe deeper into the methodology or findings
        2. Address potential limitations or gaps
        3. Explore implications or applications of the research
        4. Connect this research to broader scientific contexts
        5. Suggest future research directions

        Format each question as a separate numbered point.

        Article:
        {article_text[:5000]}
        """

        return client.generate_text(prompt)

    def compare_prompt_techniques(self, article_text, techniques=None):
        """
        Compares different prompt engineering techniques on the same article
        Returns a dictionary with results from each technique
        """
        if techniques is None:
            techniques = {
                "Basic": "Summarize this scientific article",
                "Specific": "Summarize this scientific article, focusing on the methodology and findings",
                "Role-based": "You are a scientific research assistant helping a professor. Summarize this article highlighting key contributions",
                "Step-by-step": "First, identify the research question. Second, describe the methodology. Third, explain the results. Finally, summarize the conclusions.",
                "Few-shot": "Example summary 1: [Paper title] investigated [topic] using [method] and found [result].\nExample summary 2: The researchers explored [topic] through [method], revealing [result].\nNow summarize this article in a similar style:"
            }

        results = {}

        for name, prompt_template in techniques.items():
            full_prompt = f"{prompt_template}\n\nArticle:\n{article_text[:3000]}"

            try:
                results[name] = client.generate_text(full_prompt)
            except Exception as e:
                results[name] = f"Error: {str(e)}"

        return results

    def get_citation_recommendation(self, article_text):
        """Recommends how to cite this article based on its content"""

        prompt = """
        Based on the content of this scientific article, generate:
        1. An APA style citation (assume current year if publication date isn't mentioned)
        2. Three key points that would be most relevant to cite from this paper
        3. Potential fields or topics where citing this article would be appropriate

        Article:
        """ + article_text[:4000]

        return client.generate_text(prompt)

    def summarize_with_visuals(self, article_text):
        """
        Provides guidance on what visual elements would complement a summary
        of this article (diagrams, charts, etc.)
        """

        prompt = """
        Based on this scientific article, recommend 3-5 visual elements (diagrams,
        charts, figures) that would best complement a summary of this research.

        For each recommended visual:
        1. Describe what the visual should show
        2. Explain why this visualization would be valuable
        3. Suggest a title and caption

        Article:
        """ + article_text[:4000]

        return client.generate_text(prompt)

# Create a simple demo UI with widgets
def create_summarizer_ui():
    """Creates an interactive UI for the scientific article summarizer"""
    # Create the summarizer
    summarizer = ScientificArticleSummarizer()

    # Create widgets
    article_input = widgets.Textarea(
        value='',
        placeholder='Paste scientific article text here',
        description='Article:',
        layout={'width': '100%', 'height': '200px'}
    )

    audience_dropdown = widgets.Dropdown(
        options=['general', 'undergraduate', 'graduate', 'expert'],
        value='undergraduate',
        description='Audience:',
        layout={'width': '50%'}
    )

    focus_checkboxes = widgets.SelectMultiple(
        options=['main findings', 'methodology', 'significance', 'limitations', 'future research', 'practical implications'],
        value=['main findings', 'methodology', 'significance'],
        description='Focus on:',
        layout={'width': '50%'}
    )

    max_length_slider = widgets.IntSlider(
        value=500,
        min=100,
        max=1000,
        step=50,
        description='Max length:',
        layout={'width': '50%'}
    )

    summarize_button = widgets.Button(
        description='Summarize Article',
        button_style='primary',
        icon='file-text'
    )

    extract_button = widgets.Button(
        description='Extract Key Elements',
        button_style='info',
        icon='list'
    )

    questions_button = widgets.Button(
        description='Generate Questions',
        button_style='success',
        icon='question'
    )

    compare_button = widgets.Button(
        description='Compare Prompt Techniques',
        button_style='warning',
        icon='random'
    )

    output_area = widgets.Output(
        layout={'border': '1px solid #ddd', 'min_height': '200px', 'width': '100%', 'padding': '10px'}
    )

    # Define button actions
    def on_summarize_clicked(b):
        with output_area:
            output_area.clear_output()
            print("Generating summary...")
            summary = summarizer.summarize_article(
                article_input.value,
                audience_level=audience_dropdown.value,
                focus_areas=list(focus_checkboxes.value),
                max_length=max_length_slider.value
            )
            output_area.clear_output()
            display(Markdown(summary))

    def on_extract_clicked(b):
        with output_area:
            output_area.clear_output()
            print("Extracting key elements...")
            elements = summarizer.extract_key_elements(article_input.value)
            output_area.clear_output()

            if isinstance(elements, dict) and "error" not in elements:
                for key, value in elements.items():
                    display(Markdown(f"**{key}**: {value}"))
            else:
                print(elements)

    def on_questions_clicked(b):
        with output_area:
            output_area.clear_output()
            print("Generating follow-up questions...")
            questions = summarizer.generate_follow_up_questions(article_input.value)
            output_area.clear_output()

            if isinstance(questions, list):
                for i, question in enumerate(questions, 1):
                    display(Markdown(f"{i}. {question}"))
            else:
                display(Markdown(questions))

    def on_compare_clicked(b):
        with output_area:
            output_area.clear_output()
            print("Comparing prompt techniques...")
            results = summarizer.compare_prompt_techniques(article_input.value)
            output_area.clear_output()

            for technique, result in results.items():
                display(Markdown(f"### Technique: {technique}"))
                display(Markdown(result))
                display(Markdown("---"))

    # Connect buttons to actions
    summarize_button.on_click(on_summarize_clicked)
    extract_button.on_click(on_extract_clicked)
    questions_button.on_click(on_questions_clicked)
    compare_button.on_click(on_compare_clicked)

    # Create tabs for different functionalities
    tab_summarize = widgets.VBox([
        widgets.HBox([audience_dropdown, max_length_slider]),
        focus_checkboxes,
        summarize_button
    ])

    tab_analyze = widgets.VBox([
        extract_button,
        questions_button,
        compare_button
    ])

    tabs = widgets.Tab(children=[tab_summarize, tab_analyze])
    tabs.set_title(0, 'Summarize')
    tabs.set_title(1, 'Analyze')

    # Assemble the complete UI
    ui = widgets.VBox([
        widgets.HTML("<h2>Scientific Article Summarizer</h2>"),
        article_input,
        tabs,
        widgets.HTML("<h3>Output:</h3>"),
        output_area
    ])

    return ui

## Step 6: Try it!

Copy in the text from a scientific article (or just generate one below)

In [17]:
client.generate_text("Make up a short scientific paper")

"## The Impact of Mimicking Avian Coloration on Dragonfly Predation: An Experimental Study\n\n**Abstract:**\n\nMany insect species utilize coloration as a defense mechanism against predation. This study investigates the potential for non-avian insects, specifically dragonflies, to be deterred by coloration patterns mimicking those found in birds. We hypothesized that dragonflies would exhibit reduced predation rates on prey models painted with colors and patterns reminiscent of common bird species, compared to prey models with neutral or disruptive coloration. Our results suggest a significant, albeit preliminary, trend supporting this hypothesis, warranting further investigation into the role of avian mimicry in insect defense against dragonfly predation.\n\n**Introduction:**\n\nInsect coloration plays a crucial role in survival, serving functions ranging from camouflage and thermoregulation to aposematism (warning coloration). While aposematism is traditionally associated with bright

In [16]:
# To run the interactive UI in a notebook:
from IPython.display import display

ui = create_summarizer_ui()
display(ui)